<a href="https://colab.research.google.com/github/maulanaakbardj/notebook_TG/blob/main/TG_3_AML_FRAUD_ex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# -

In [ ]:
!pip install pyTigerGraph
import pyTigerGraph as tg

In [ ]:
conn = tg.TigerGraphConnection(host="https://#your_subdomain#.i.tgcloud.io", password="#your_pass")

In [ ]:
print(conn.gsql('''
CREATE VERTEX Negara (PRIMARY_ID id STRING) WITH primary_id_as_attribute="true"
CREATE VERTEX Pengguna (PRIMARY_ID id STRING) WITH primary_id_as_attribute="true"
CREATE VERTEX Akun (PRIMARY_ID id STRING, init_balance DOUBLE, account_type STRING, tx_behavior INT, pagerank FLOAT, label INT, current_balance DOUBLE, min_send_tx DOUBLE, min_recieve_tx DOUBLE, max_send_tx DOUBLE, max_recieve_tx DOUBLE, avg_send_tx DOUBLE, avg_recieve_tx DOUBLE, cnt_recieve_tx INT, cnt_send_tx INT) WITH primary_id_as_attribute="true"
CREATE VERTEX Transaksi (PRIMARY_ID id STRING, tx_behavior_id INT, amount DOUBLE, is_fraud BOOL) WITH primary_id_as_attribute="true"
CREATE VERTEX Alert (PRIMARY_ID id STRING, alert_type STRING, ts INT) WITH primary_id_as_attribute="true"
CREATE UNDIRECTED EDGE Based_In (From Pengguna, To Negara)
CREATE UNDIRECTED EDGE Customer_Account (From Pengguna, To Akun)
CREATE UNDIRECTED EDGE Transaction_Flagged (From Transaksi, To Alert)
CREATE DIRECTED EDGE Send_To (From Akun, To Akun) WITH REVERSE_EDGE="reverse_Send_To"
CREATE DIRECTED EDGE Send_Transaction (From Akun, To Transaksi, ts INT, tx_type STRING) WITH REVERSE_EDGE="reverse_Send_Transaction"
CREATE DIRECTED EDGE Recieve_Transaction (From Transaksi, To Akun, ts INT, tx_type STRING) WITH REVERSE_EDGE="reverse_Recieve_Transaction"
''', options=[]))

Successfully created vertex types: [Negara].
Successfully created vertex types: [Pengguna].
Successfully created vertex types: [Akun].
Successfully created vertex types: [Transaksi].
Successfully created vertex types: [Alert].
Successfully created edge types: [Based_In].
Successfully created edge types: [Customer_Account].
Successfully created edge types: [Transaction_Flagged].
Successfully created edge types: [Send_To].
Successfully created reverse edge types: [reverse_Send_To].
Successfully created edge types: [Send_Transaction].
Successfully created reverse edge types: [reverse_Send_Transaction].
Successfully created edge types: [Recieve_Transaction].
Successfully created reverse edge types: [reverse_Recieve_Transaction].


In [ ]:
print(conn.gsql('''CREATE GRAPH AMLSim(Negara, Pengguna, Akun, Transaksi, Alert, Based_In, Customer_Account, Transaction_Flagged, Send_To, Send_Transaction, reverse_Send_Transaction, Recieve_Transaction, reverse_Recieve_Transaction)''', options=[]))

The graph AMLSim is created.


In [ ]:
import pandas as pd
!git clone https://github.com/TigerGraph-OSS/AMLSim_Python_Lab

Cloning into 'AMLSim_Python_Lab'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 84 (delta 26), reused 48 (delta 10), pack-reused 0
Unpacking objects: 100% (84/84), done.


In [ ]:
accounts = pd.read_csv("AMLSim_Python_Lab/data/accounts.csv", sep=",")
alerts = pd.read_csv("AMLSim_Python_Lab/data/alerts.csv", sep=",")
transactions = pd.read_csv("AMLSim_Python_Lab/data/transactions.csv", sep=",")

In [ ]:
accounts

,ACCOUNT_ID,CUSTOMER_ID,INIT_BALANCE,COUNTRY,ACCOUNT_TYPE,IS_FRAUD,TX_BEHAVIOR_ID
0,0,C_0,184.44,US,I,False,1
1,1,C_1,175.80,US,I,False,1
2,2,C_2,142.06,US,I,False,1
3,3,C_3,125.89,US,I,False,1
4,4,C_4,151.13,US,I,False,1
...,...,...,...,...,...,...,...
9995,9995,C_9995,394.35,US,I,True,5
9996,9996,C_9996,344.98,US,I,True,5
9997,9997,C_9997,260.85,US,I,True,5
9998,9998,C_9998,323.79,US,I,True,5


In [ ]:
alerts

,ALERT_ID,ALERT_TYPE,IS_FRAUD,TX_ID,SENDER_ACCOUNT_ID,RECEIVER_ACCOUNT_ID,TX_TYPE,TX_AMOUNT,TIMESTAMP
0,193,fan_in,True,82,6976,9739,TRANSFER,4.85,0
1,377,cycle,True,949,5776,2570,TRANSFER,10.27,0
2,189,fan_in,True,6280,9999,9530,TRANSFER,2.74,1
3,377,cycle,True,7999,1089,7352,TRANSFER,10.27,1
4,130,fan_in,True,12975,7025,9708,TRANSFER,3.53,2
...,...,...,...,...,...,...,...,...,...
1714,272,cycle,True,1316271,2465,707,TRANSFER,16.31,198
1715,44,fan_in,True,1316636,1453,8709,TRANSFER,2.81,199
1716,222,cycle,True,1317852,9446,790,TRANSFER,11.64,199
1717,74,fan_in,True,1322020,6969,9529,TRANSFER,3.86,199


In [ ]:
transactions

,TX_ID,SENDER_ACCOUNT_ID,RECEIVER_ACCOUNT_ID,TX_TYPE,TX_AMOUNT,TIMESTAMP,IS_FRAUD,ALERT_ID
0,1,6456,9069,TRANSFER,465.05,0,False,-1
1,2,7516,9543,TRANSFER,564.64,0,False,-1
2,3,2445,9356,TRANSFER,598.94,0,False,-1
3,4,2576,4617,TRANSFER,466.07,0,False,-1
4,5,3524,1773,TRANSFER,405.63,0,False,-1
...,...,...,...,...,...,...,...,...
1323229,1323230,3733,8051,TRANSFER,112.98,199,False,-1
1323230,1323231,2536,8732,TRANSFER,459.64,199,False,-1
1323231,1323232,1466,8586,TRANSFER,468.60,199,False,-1
1323232,1323233,1451,3849,TRANSFER,562.36,199,False,-1


In [ ]:
accounts['ACCOUNT_ID'] = accounts['ACCOUNT_ID'].astype(str)
alerts['ALERT_ID'] = alerts['ALERT_ID'].astype(str)
alerts['SENDER_ACCOUNT_ID'] = alerts['SENDER_ACCOUNT_ID'].astype(str)
alerts['RECEIVER_ACCOUNT_ID'] = alerts['RECEIVER_ACCOUNT_ID'].astype(str)
transactions['TX_ID'] = transactions['TX_ID'].astype(str)
transactions['SENDER_ACCOUNT_ID'] = transactions['SENDER_ACCOUNT_ID'].astype(str)
transactions['RECEIVER_ACCOUNT_ID'] = transactions['RECEIVER_ACCOUNT_ID'].astype(str)

In [ ]:
conn.graphname = "AMLSim"
conn.apiToken = conn.getToken(conn.createSecret())

In [ ]:
# UPSERT VERTEX "Pengguna"
v_customer = conn.upsertVertexDataFrame(accounts, "Pengguna", "CUSTOMER_ID", attributes={"id": "CUSTOMER_ID"})
print(str(v_customer) + " Customer VERTICES Upserted")
# UPSERT VERTEX "Akun"
v_account = conn.upsertVertexDataFrame(accounts, "Akun", "ACCOUNT_ID", attributes={"id": "ACCOUNT_ID", "init_balance": "INIT_BALANCE", "account_type": "ACCOUNT_TYPE", "tx_behavior": "TX_BEHAVIOR_ID"})
print(str(v_account) + " Account VERTICES Upserted")
# UPSERT VERTEX "Transaksi"
v_transaction = conn.upsertVertexDataFrame(transactions, "Transaksi", "TX_ID", attributes={"id": "TX_ID", "amount": "TX_AMOUNT", "is_fraud": "IS_FRAUD"})
print(str(v_transaction) + " Transaction VERTICES Upserted")
# UPSERT VERTEX "Alert"
v_alert = conn.upsertVertexDataFrame(alerts, "Alert", "ALERT_ID", attributes={"id": "ALERT_ID", "alert_type": "ALERT_TYPE", "ts": "TIMESTAMP"})
print(str(v_alert) + " Alert VERTICES Upserted")
# UPSERT EDGE "Send_Transaction"
e_send_transaction = conn.upsertEdgeDataFrame(transactions, "Akun", "Send_Transaction", "Transaksi", from_id="SENDER_ACCOUNT_ID", to_id="TX_ID", attributes={"ts": "TIMESTAMP", "tx_type": "TX_TYPE"})
print(str(e_send_transaction) + " Send_Transaction EDGES Upserted")
# UPSERT EDGE "Send_Transaction"
e_recieve_transaction = conn.upsertEdgeDataFrame(transactions, "Transaksi", "Recieve_Transaction", "Akun", from_id="TX_ID", to_id="RECEIVER_ACCOUNT_ID", attributes={"ts": "TIMESTAMP", "tx_type": "TX_TYPE"})
print(str(e_recieve_transaction) + " Recieve_Transaction EDGES Upserted")
# UPSERT EDGE "Send_To"
e_send_to = conn.upsertEdgeDataFrame(transactions, "Akun", "Send_To", "Akun", from_id="SENDER_ACCOUNT_ID", to_id="RECEIVER_ACCOUNT_ID", attributes={})
print(str(e_send_to) + " Send_To EDGES Upserted")
# UPSERT EDGE "Transaction_Flagged"
e_transaction_flagged = conn.upsertEdgeDataFrame(alerts, "Transaksi", "Transaction_Flagged", "Alert", from_id="TX_ID", to_id="ALERT_ID", attributes={})
print(str(e_transaction_flagged) + " Transaction_Flagged EDGES Upserted")
# UPSERT EDGE "Customer_Account"
e_customer_account = conn.upsertEdgeDataFrame(accounts, "Pengguna", "Customer_Account", "Akun", from_id="CUSTOMER_ID", to_id="ACCOUNT_ID", attributes={})
print(str(e_customer_account) + " Customer_Account EDGES Upserted")
# UPSERT EDGE "Based_In"
e_based_in = conn.upsertEdgeDataFrame(accounts, "Pengguna", "Based_In", "Negara", from_id="CUSTOMER_ID", to_id="COUNTRY", attributes={})
print(str(e_based_in) + " Based_In EDGES Upserted")
# PRINT OUT STATS
print("=====TOTAL_UPSERTS=====")
print(str(v_customer+v_account+v_transaction+v_alert) + " TOTAL VERTICES")
print(str(e_send_transaction+e_recieve_transaction+e_send_to+e_transaction_flagged+e_customer_account+e_based_in) + " TOTAL EDGES")

10000 Customer VERTICES Upserted
10000 Account VERTICES Upserted
1323234 Transaction VERTICES Upserted
391 Alert VERTICES Upserted
1323234 Send_Transaction EDGES Upserted
1323234 Recieve_Transaction EDGES Upserted
68947 Send_To EDGES Upserted
1719 Transaction_Flagged EDGES Upserted
10000 Customer_Account EDGES Upserted
10000 Based_In EDGES Upserted
=====TOTAL_UPSERTS=====
1343625 TOTAL VERTICES
2737134 TOTAL EDGES


# --



In [ ]:
print(conn.gsql('''
CREATE QUERY accountActivity() FOR GRAPH AMLSim {
SumAccum<DOUBLE> @s_sumAmt, @r_sumAmt;
SumAccum<DOUBLE> @s_txCnt, @r_txCnt;
MinAccum<DOUBLE> @s_minAmt, @r_minAmt;
MaxAccum<DOUBLE> @s_maxAmt, @r_maxAmt;
AvgAccum @s_avgAmt, @r_avgAmt;
Seed = {Akun.*};
acctSend = SELECT tgt FROM Seed:s -(Send_Transaction:e)-> Transaksi:tgt
ACCUM s.@s_sumAmt += tgt.amount, s.@s_txCnt += 1, s.@s_minAmt += tgt.amount, s.@s_maxAmt += tgt.amount, s.@s_avgAmt += tgt.amount
POST-ACCUM s.current_balance = s.@s_sumAmt - s.init_balance, s.min_send_tx = s.@s_minAmt, s.max_send_tx = s.@s_maxAmt, s.avg_send_tx = s.@s_avgAmt, s.cnt_send_tx = s.@s_txCnt;
acctRecieve = SELECT tgt FROM Seed:s -(reverse_Recieve_Transaction:e)-> Transaksi:tgt
ACCUM s.@r_sumAmt += tgt.amount, s.@r_txCnt += 1, s.@r_minAmt += tgt.amount, s.@r_maxAmt += tgt.amount, s.@r_avgAmt += tgt.amount
POST-ACCUM s.current_balance = s.@r_sumAmt + s.init_balance, s.min_recieve_tx = s.@r_minAmt, s.max_recieve_tx = s.@r_maxAmt, s.avg_recieve_tx = s.@r_avgAmt, s.cnt_recieve_tx = s.@r_txCnt;
PRINT "Features Have Been Calculated";
}
INSTALL QUERY accountActivity
''', options=[]))

Semantic Check Fails: The query name accountActivity is used by another object! Please use a different name.
Failed to create queries: [accountActivity].


In [ ]:
print(conn.gsql('''
CREATE QUERY label_prop (SET<STRING> v_type, SET<STRING> e_type, INT max_iter, INT output_limit, BOOL print_accum = TRUE, STRING file_path = "", STRING attr = "") FOR GRAPH AMLSim{
OrAccum @@changed = true;
MapAccum<INT, INT> @map;
MapAccum<INT, INT> @@commSizes;
SumAccum<INT> @label, @num;
FILE f (file_path);
Start = {v_type};
Start = SELECT s FROM Start:s ACCUM s.@label = getvid(s);
WHILE @@changed == true LIMIT max_iter DO
@@changed = false;
Start = SELECT s
FROM Start:s -(e_type:e)-> :t
ACCUM t.@map += (s.@label -> 1)
POST-ACCUM INT maxV = 0, INT label = 0,
FOREACH (k,v) IN t.@map DO
CASE WHEN v > maxV THEN maxV = v, label = k
END
END,
CASE WHEN label != 0 AND t.@label != label AND maxV > t.@num THEN
@@changed += true,
t.@label = label,
t.@num = maxV
END,
t.@map.clear();
END;
Start = {v_type};
Start =  SELECT s FROM Start:s
POST-ACCUM
IF attr != "" THEN s.setAttr(attr, s.@label) END,
IF file_path != "" THEN f.println(s, s.@label) END,
IF print_accum THEN @@commSizes += (s.@label -> 1) END
LIMIT output_limit;
IF print_accum THEN
PRINT @@commSizes;
PRINT Start[Start.@label];
END;
}
INSTALL QUERY label_prop
''', options=[]))

Semantic Check Fails: The query name label_prop is used by another object! Please use a different name.
Failed to create queries: [label_prop].


In [ ]:
print(conn.gsql('''
CREATE QUERY pageRank (STRING v_type, STRING e_type, FLOAT max_change=0.001, INT max_iter=25, FLOAT damping=0.85, INT top_k = 100, BOOL print_accum = TRUE, STRING result_attr =  "", STRING file_path = "", BOOL display_edges = FALSE) FOR GRAPH AMLSim {
TYPEDEF TUPLE<VERTEX Vertex_ID, FLOAT score> Vertex_Score;
HeapAccum<Vertex_Score>(top_k, score DESC) @@topScores;
MaxAccum<FLOAT> @@max_diff = 9999;
SumAccum<FLOAT> @recvd_score = 0;
SumAccum<FLOAT> @score = 1;
SetAccum<EDGE> @@edgeSet;
FILE f (file_path);
Start = {v_type};
WHILE @@max_diff > max_change LIMIT max_iter DO
@@max_diff = 0;
V = SELECT s FROM Start:s -(e_type:e)-> v_type:t
ACCUM t.@recvd_score += s.@score/(s.outdegree(e_type))
POST-ACCUM s.@score = (1.0-damping) + damping * s.@recvd_score, s.@recvd_score = 0, @@max_diff += abs(s.@score - s.@score');
END; 
IF file_path != "" THEN
f.println("Vertex_ID", "PageRank");
END;
V = SELECT s FROM Start:s
POST-ACCUM
IF result_attr != "" THEN s.setAttr(result_attr, s.@score) END,
IF file_path != "" THEN f.println(s, s.@score) END,
IF print_accum THEN @@topScores += Vertex_Score(s, s.@score) END;
IF print_accum THEN PRINT @@topScores;
IF display_edges THEN PRINT Start[Start.@score];
Start = SELECT s FROM Start:s -(e_type:e)-> v_type:t
ACCUM @@edgeSet += e;
PRINT @@edgeSet;
END; END;
}
INSTALL QUERY pageRank
''', options=[]))

Semantic Check Fails: The query name pageRank is used by another object! Please use a different name.
Failed to create queries: [pageRank].


In [ ]:
print(conn.gsql('''
CREATE QUERY txMultiHopLimit() FOR GRAPH AMLSim SYNTAX v2 {  
/* This grabs the first 10,000 transactions. The beginning has all of the fraudulent transactions and the latter part has some non-fraudulent transactions. */
TYPEDEF TUPLE <BOOL tx_fraud, DOUBLE tx_amount, FLOAT s_pagerank, INT s_label, DOUBLE s_min_send_tx, DOUBLE s_min_receieve_tx, DOUBLE s_max_send_tx, DOUBLE s_max_recieve_tx, DOUBLE s_avg_send_tx, DOUBLE s_avg_recieve_tx, INT s_cnt_recieve_tx, INT s_cnt_send_tx, INT s_timestamp, FLOAT r_pagerank, INT r_label, DOUBLE r_min_send_tx, DOUBLE r_min_receieve_tx, DOUBLE r_max_send_tx, DOUBLE r_max_recieve_tx, DOUBLE r_avg_send_tx, DOUBLE r_avg_recieve_tx, INT r_cnt_recieve_tx, INT r_cnt_send_tx, INT r_timestamp> ORDER_TX;
HeapAccum<ORDER_TX>(10000, tx_fraud DESC) @@txRecords;
Seed = {Akun.*};
acctSend = SELECT tgt FROM Seed:s - ((Send_Transaction>):e1) - Transaksi:v1 - ((Recieve_Transaction>):e2) - Akun:tgt 
ACCUM @@txRecords += ORDER_TX(v1.is_fraud, v1.amount, s.pagerank, s.label, s.min_send_tx, s.min_recieve_tx, s.max_send_tx, s.max_recieve_tx, s.avg_send_tx, s.avg_recieve_tx, s.cnt_recieve_tx, s.cnt_send_tx, e1.ts, tgt.pagerank, tgt.label, tgt.min_send_tx, tgt.min_recieve_tx, tgt.max_send_tx, tgt.max_recieve_tx, tgt.avg_send_tx, tgt.avg_recieve_tx, tgt.cnt_recieve_tx, tgt.cnt_send_tx, e2.ts);  
PRINT @@txRecords; 
}
INSTALL QUERY txMultiHopLimit
''', options=[]))

Successfully created queries: [txMultiHopLimit3].
Start installing queries, about 1 minute ...
txMultiHopLimit3 query: curl -X GET 'https://127.0.0.1:9000/query/AMLSim/txMultiHopLimit3'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.
Select 'm1' as compile server, now connecting ...
Node 'm1' is prepared as compile server.

Query installation failed!


# ---

In [ ]:
accAct = conn.runInstalledQuery("accountActivity", {})

In [ ]:
pagerank = conn.runInstalledQuery("pageRank", {"v_type":"Akun", "e_type":"Send_To", "max_change":"0.001", "max_iter":"25", "damping":"0.85", "top_k":"10", "print_accum":"TRUE", "result_attr":"pagerank", "file_path":"", "display_edges":"FALSE"})

In [ ]:
label_prop = conn.runInstalledQuery("label_prop", {"v_type":"Akun", "e_type":"Send_To", "max_iter":"10", "output_limit":"10", "print_accum":"TRUE", "file_path":"", "attr":"label"})

In [ ]:
tx_hop2 = conn.runInstalledQuery("txMultiHopLimit3", {}, timeout="99999999999", sizeLimit="1500000000")

In [ ]:
!pip install flat-table
import flat_table
df_tx_hop = pd.DataFrame(tx_hop2[0]["@@txRecords"])
df_tx_hop = flat_table.normalize(df_tx_hop)
df_tx_hop = df_tx_hop.drop(columns=["index"])
df_tx_hop.to_csv("tg_fraud_data.csv", index=False)

In [ ]:
df = pd.read_csv("/content/tg_fraud_data.csv")
df

,tx_fraud,tx_amount,s_pagerank,s_label,s_min_send_tx,s_min_receieve_tx,s_max_send_tx,s_max_recieve_tx,s_avg_send_tx,s_avg_recieve_tx,s_cnt_recieve_tx,s_cnt_send_tx,s_timestamp,r_pagerank,r_label,r_min_send_tx,r_min_receieve_tx,r_max_send_tx,r_max_recieve_tx,r_avg_send_tx,r_avg_recieve_tx,r_cnt_recieve_tx,r_cnt_send_tx,r_timestamp
0,True,14.10,1.73309,317718754,14.10,14.10,19283946.00,13447807.00,3.293691e+06,110284.49535,198,25,115,1.32812,248512844,4.00,11.54,10994822.00,21474836.47,7.524456e+05,574262.20641,39,20,115
1,True,3.01,0.82673,244318217,3.01,8.23,485.33,14765216.00,4.826653e+02,111138.31090,145,181,88,2.35695,312475745,12.33,3.01,16.79,19464572.00,1.235134e+01,103307.23976,331,209,88
2,True,4.40,5.47480,297795753,4.40,8.34,537.96,21474836.47,5.350122e+02,95984.24226,345,181,63,5.10034,293601296,2232.67,4.34,6077223.50,21474836.47,1.077381e+06,386934.16786,215,15,63
3,True,10.69,1.30538,279969972,10.69,10.69,15.38,345.89,1.534894e+01,132.01391,87,151,45,0.84428,244318328,3.47,10.69,21474836.47,21474836.47,3.017031e+06,777829.75438,73,19,45
4,True,12.81,0.73438,244318413,12.81,10.22,15.23,21474836.47,1.521929e+01,292459.80867,75,226,47,0.77667,261095626,12.81,12.81,21474836.47,1256044.37,5.027736e+06,43182.16833,48,17,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,False,174.55,1.07258,267386884,16.36,7.86,174.55,17937276.00,1.735132e+02,178590.28625,112,302,123,3.35073,244318243,3.87,3.03,9.01,19628146.00,8.989920e+00,64217.55408,309,256,123
9996,False,8.23,1.64001,340787375,8.23,9.49,8.23,11067100.00,8.230000e+00,56579.23990,201,380,140,2.10358,348127485,434.86,5.79,434.86,11295013.00,4.348600e+02,69182.50646,175,180,140
9997,False,174.55,1.07258,267386884,16.36,7.86,174.55,17937276.00,1.735132e+02,178590.28625,112,302,3,0.49442,255852569,12.63,102.02,12.63,11732449.00,1.263000e+01,186141.94226,124,209,3
9998,False,174.55,1.07258,267386884,16.36,7.86,174.55,17937276.00,1.735132e+02,178590.28625,112,302,93,0.71921,348127558,341.54,17.38,341.54,21474836.47,3.415400e+02,389021.36434,145,21,93


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   tx_fraud           10000 non-null  bool   
 1   tx_amount          10000 non-null  float64
 2   s_pagerank         10000 non-null  float64
 3   s_label            10000 non-null  int64  
 4   s_min_send_tx      10000 non-null  float64
 5   s_min_receieve_tx  10000 non-null  float64
 6   s_max_send_tx      10000 non-null  float64
 7   s_max_recieve_tx   10000 non-null  float64
 8   s_avg_send_tx      10000 non-null  float64
 9   s_avg_recieve_tx   10000 non-null  float64
 10  s_cnt_recieve_tx   10000 non-null  int64  
 11  s_cnt_send_tx      10000 non-null  int64  
 12  s_timestamp        10000 non-null  int64  
 13  r_pagerank         10000 non-null  float64
 14  r_label            10000 non-null  int64  
 15  r_min_send_tx      10000 non-null  float64
 16  r_min_receieve_tx  1000

In [ ]:
df.tx_fraud.unique()

array([ True, False])

In [ ]:
df.tx_fraud.value_counts()

False    8281
True     1719
Name: tx_fraud, dtype: int64

# ----